# Assignment 2: Naive Bayes [20 marks]

Student Name: Cheah Jia Huei

Student ID: 1078203

## General info

<b>Due date</b>: Friday, 2 September 2022, 5pm

<b>Submission method</b>: Canvas submission

<b>Submission materials</b>: completed copy of this iPython notebook

<b>Late submissions</b>: -10% per day up to 5 days (both weekdays and weekends count)
<ul>
    <li>one day late, -2.0;</li>
    <li>two days late, -4.0;</li>
    <li>three days late, -6.0;</li>
    <li>four days late, -8.0;</li>
    <li>five days late, -10.0;</li>
</ul>

<b>Marks</b>: 20% of mark for class. 

<b>Materials</b>: See [Using Jupyter Notebook and Python page](https://canvas.lms.unimelb.edu.au/courses/126693/pages/python-and-jupyter-notebooks?module_item_id=3950453) on Canvas (under Modules> Coding Resources) for information on the basic setup required for this class, including an iPython notebook viewer.If your iPython notebook doesn't run on the marker's machine, you will lose marks. <b> You should use Python 3</b>.  


<b>Evaluation</b>: Your iPython notebook should run end-to-end without any errors in a reasonable amount of time, and you must follow all instructions provided below, including specific implementation requirements and instructions for what needs to be printed (please avoid printing output we don't ask for). You should implement functions for the skeletons listed below. You may implement any number of additional (helper) functions. You should leave the output from running your code in the iPython notebook you submit, to assist with marking. The amount each section is worth is given in parenthesis after the instructions. 

You will be marked not only on the correctness of your methods, but also the quality and efficiency of your code: in particular, you should be careful to use Python built-in functions and operators when appropriate and pick descriptive variable names that adhere to <a href="https://www.python.org/dev/peps/pep-0008/">Python style requirements</a>. If you think it might be unclear what you are doing, you should comment your code to help the marker make sense of it. We reserve the right to deduct up to 4 marks for unreadable or excessively inefficient code.

7 of the marks available for this Project will be assigned to whether the five specified Python functions work in a manner consistent with the materials from COMP90049. Any other implementation will not be directly assessed (except insofar as it is required to make these five functions work correctly).

13 of the marks will be assigned to your responses to the questions, in terms of both accuracy and insightfulness. We will be looking for evidence that you have an implementation that allows you to explore the problem, but also that you have thought deeply about the data and the behaviour of the Naive Bayes classifier.

<b>Updates</b>: Any major changes to the assignment will be announced via Canvas. Minor changes and clarifications will be announced on the discussion board (Piazza -> Assignments -> A2); we recommend you check it regularly.

<b>Academic misconduct</b>: While you may discuss this homework in general terms with other students, it ultimately is still an individual task. Reuse of code or other instances of clear influence will be considered cheating. Please check the <a href="https://canvas.lms.unimelb.edu.au/courses/126693/modules#module_734188">CIS Academic Honesty training</a> for more information. We will be checking submissions for originality and will invoke the University’s <a href="http://academichonesty.unimelb.edu.au/policy.html">Academic Misconduct policy</a> where inappropriate levels of collusion or plagiarism are deemed to have taken place.

**IMPORTANT**

Please carefully read and fill out the <b>Authorship Declaration</b> form at the bottom of the page. Failure to fill out this form results in the following deductions: 
<UL TYPE=”square”>
<LI>missing Authorship Declaration at the bottom of the page, -10.0
<LI>incomplete or unsigned Authorship Declaration at the bottom of the page, -5.0
</UL>

## Part 1: Base code [7 marks]

Instructions
1. Do **not** shuffle the data set
2. Treat the features as nominal and use them as provided (e.g., do **not** convert them to other feature types, such as numeric ones). Implement a Naive Bayes classifier with appropriate likelihood function for the data.
3. You should implement the Naive Bayes classifier from scratch. Do **not** use existing implementations/learning algorithms. You must use epsilon smoothing strategy as discussed in the Naive Bayes lecture. 
4. Apart from the instructions in point 3, you may use libraries to help you with data reading, representation, maths or evaluation.
5. Ensure that all and only required information is printed, as indicated in the final three code cells. Failure to adhere to print the required information will result in **[-1 mark]** per case. *(We don't mind details like you print a list or several numbers -- just make sure the information is displayed so that it's easily accessible)*
6. Please place the jupyter notebook into the same folder as the input data.


In [ ]:
# This function should open a csv file and read the data into a useable format [0.5 mark]

def preprocess(filename):
    X = list()
    label = list()
    X_and_label = list()
    data = open(filename).readlines()
    temp_col = data[0].strip().split(",")
    feat_headers = temp_col[1:]
    for i in range(1,len(data)):
        splitted_data = data[i].strip().split(",")
        X_and_label.append(splitted_data[1:])
        X.append(splitted_data[1:-1])
        label.append(splitted_data[-1])
    return (X, label,feat_headers, X_and_label)

In [ ]:
# This function should build a supervised NB model [3 marks]
# the X and label inputs here should belong to the training set
import numpy as np
import math
def train(X,label,X_and_label):
    # use epsilon sommothing strategy
    feature_vals = list()
    # get the set of all feature labels
    number_of_features = len(X[0])
    for j in range(0,number_of_features):
       poss_feat_val = list(set([instance[j] for instance in X]))
       feature_vals.append(poss_feat_val)
    # get the set of all labels
    label_types = list(set(label))
    # get the total number of labels
    label_total = len(label)
    # count the number of occurrences of each label
    label_occs = list()
    for element in label_types:
        label_num = label.count(element)
        label_occs.append(label_num)
    # convert label_types and their respective occurrences into a dictionary
    # this is used in the process of counting P(x|y)
    label_count_dict = dict(zip(label_types,label_occs))
    label_prob = [occ/label_total for occ in label_occs]
    label_prob_dict = dict(zip(label_types,label_prob))
    # how to find two element occurring at the same time in a row
    # helper function to find P(x|y) of all features
    # inner function can access all variables from outer function
    def conditional_prob(X,label):
        
        # create an empty dictionary whih records all of the probability patterns
        Y_copy = np.array(X_and_label)
        feature_probs = dict()
        # outer loop over all label types
        for indi_lab in label_types:
            # second and third loop for the 2d feature_vals array
            for k in range(len(feature_vals)):
                for m in range(len(feature_vals[k])):
                    result = Y_copy[np.where( (Y_copy[:,k] == feature_vals[k][m]) & (Y_copy[:,-1] == indi_lab))]
                    prob_numerator = len(result)
                    prob_denomitor = label_count_dict[indi_lab]
                    cond_prob = prob_numerator / prob_denomitor
                    # choose val of epsiolon: 1 * 10^-5
                    epsilon = math.pow(10,-5)
                    if cond_prob == 0:
                        cond_prob = epsilon
                    # The key of the feature_probs dictionary: featureNum_featureVal_label
                    fprobs_key = str(k) +'_'+feature_vals[k][m]+'_'+indi_lab
                    feature_probs[fprobs_key] = cond_prob
        return feature_probs
    feature_probs = conditional_prob(X,label)
    return (label_prob_dict,feature_probs)


In [ ]:
# This function should predict the class for a set of instances, based on a trained model [1.5 marks]
# label_prob_dict and feature_probs were outputs from the train_model
import math
def predict(test_instance,label_prob_dict,feature_probs):
    label_types = list(label_prob_dict.keys())
    # create a probability dict for all outcomes
    cat_out = dict()
    # to avoid probability underflow, add the log probabilities
    # without the base argument in the log function, natural log computed
    for indi_label in label_types:
        # initialise the probability of that category given the test instance
        prob_cat = label_prob_dict[indi_label]
        total_cat_prob = math.log(prob_cat)
        for feat_num, feat_val in enumerate(test_instance):
            request_key = str(feat_num) + "_" + feat_val + "_" + indi_label
            if request_key in feature_probs:           
                total_cat_prob += math.log(feature_probs[request_key])
            else:
                epsilon = math.pow(10,-5)
                total_cat_prob += math.log(epsilon)
                
        cat_out[indi_label] = total_cat_prob
    max_key = max(cat_out,key=cat_out.get)
    return (cat_out,max_key)



In [ ]:
# This function should evaluate a set of predictions [1 mark]
from sklearn.metrics import f1_score, accuracy_score, precision_score, recall_score
def evaluate(X,label,label_prob_dict, feature_probs):
    # can use sklearn lib
    # working on the entire dataset for both train and test 
    # generate the prediction for the whole dataset 
    predictions = list()
    for i in range(len(X)):
        current_prediction = predict(X[i],label_prob_dict,feature_probs)
        predictions.append(current_prediction[1])
    evaluation_scores = dict()
    # get the number of labels
    number_of_labels = len(list(set(label)))
    # the true labels are 'obese' for obesity dataset and 'yes' for banking dataset
    label_types = list(set(label))
    if (number_of_labels == 2):
        evaluation_scores['f1_score'] = f1_score(label,predictions,pos_label=label_types[0],average='binary')
        evaluation_scores['precision'] = precision_score(label,predictions,pos_label=label_types[0],average='binary')
        evaluation_scores['recall'] = recall_score(label,predictions,pos_label=label_types[0],average='binary')
    elif(number_of_labels > 2):
        evaluation_scores['f1_score'] = f1_score(label,predictions,average='macro')
        evaluation_scores['precision'] = precision_score(label,predictions,average='macro')
        evaluation_scores['recall'] = recall_score(label,predictions,average='macro')
    evaluation_scores['accuracy'] = accuracy_score(label,predictions)
    evaluation_scores['error_rate'] = 1 - accuracy_score(label,predictions)
    return evaluation_scores




               


## Bank Marketing

In [ ]:
# This cell should act as your "main" function where you call the above functions 
# on the full Bank Marketing data set, and print the evaluation score. [0.33 marks]

import numpy as np

# First, read in the data and apply your NB model to the Bank Marketing data
output = preprocess('bank-marketing.csv')
X = output[0]
# After training the dataset, the label is always added to X, so create a copy using np
label = output[1]
X_and_label = output[3]
train_result = train(X,label,X_and_label)
label_prob_dict = train_result[0]
feature_probs = train_result[1]
evaluation_scores = evaluate(X,label,label_prob_dict,feature_probs)

print(evaluation_scores)






# Third, print data statistics and model predictions, as instructed below 
# N is the total number of instances, F the total number of features, L the total number of labels
# The "class probabilities" may be unnormalized

probs_3 = predict(X[-3],label_prob_dict,feature_probs)
# round the number of decimals in the "class probabilities" dictionary
class_prob_dict_3 = probs_3[0]
for key_3 in class_prob_dict_3:
    class_prob_dict_3[key_3] = round(class_prob_dict_3[key_3],4)

probs_2 = predict(X[-2],label_prob_dict,feature_probs)
# round the number of decimals in the "class probabilities" dictionary
class_prob_dict_2 = probs_2[0]
for key_2 in class_prob_dict_2:
    class_prob_dict_2[key_2] = round(class_prob_dict_2[key_2], 4)

probs_1 = predict(X[-1],label_prob_dict,feature_probs)
# round the number of decimals in the "class probabilities" dictionary
class_prob_dict_1 = probs_1[0]
for key_1 in class_prob_dict_1:
    class_prob_dict_1[key_1] = round(class_prob_dict_1[key_1], 4)
print("Feature vectors of instances [0, 1, 2]: ", X[0:3])
print("\nNumber of instances (N): ", len(X) )
print("Number of features (F): ", len(X[0]))
print("Number of labels (L): ", len(set(label)))

# should we list out the probability prediction of each class?
# N-3 is thrid instance from the end
print("\n\nPredicted class probabilities for instance N-3: ",class_prob_dict_3)
print("Predicted class for instance N-3: ", probs_3[1])
print("\nPredicted class probabilities for instance N-2: ",class_prob_dict_2 )
print("Predicted class for instance N-2: ", probs_2[1])
print("\nPredicted class probabilities for instance N-1: ", class_prob_dict_1)
print("Predicted class for instance N-1: ", probs_1[1])



## Student

In [ ]:
# This cell should act as your "main" function where you call the above functions 
# on the full Student data set, and print the evaluation score. [0.33 marks]



# First, read in the data and apply your NB model to the Student data
output = preprocess('student.csv')
X = output[0]
# After training the dataset, the label is always added to X, so create a copy using np
label = output[1]
X_and_label = output[3]
train_result = train(X,label,X_and_label)
label_prob_dict = train_result[0]

feature_probs = train_result[1]
score = evaluate(X,label,label_prob_dict,feature_probs)

probs_3 = predict(X[-3],label_prob_dict,feature_probs)
# round the number of decimals in the "class probabilities" dictionary
class_prob_dict_3 = probs_3[0]
for key_3 in class_prob_dict_3:
    class_prob_dict_3[key_3] = round(class_prob_dict_3[key_3], 4)

probs_2 = predict(X[-2],label_prob_dict,feature_probs)
# round the number of decimals in the "class probabilities" dictionary
class_prob_dict_2 = probs_2[0]
for key_2 in class_prob_dict_2:
    class_prob_dict_2[key_2] = round(class_prob_dict_2[key_2], 4)

probs_1 = predict(X[-1],label_prob_dict,feature_probs)
# round the number of decimals in the "class probabilities" dictionary
class_prob_dict_1 = probs_1[0]
for key_1 in class_prob_dict_1:
    class_prob_dict_1[key_1] = round(class_prob_dict_1[key_1], 4)


print(score)






# Third, print data statistics and model predictions, as instructed below 
# N is the total number of instances, F the total number of features, L the total number of labels
# The "class probabilities" may be unnormalized


print("Feature vectors of instances [0, 1, 2]: ", X[0:3] )

print("\nNumber of instances (N): ", len(X))
print("Number of features (F): ", len(X[0]))
print("Number of labels (L): ", len(set(label)))

print("\n\nPredicted class probabilities for instance N-3: ", class_prob_dict_3)
print("Predicted class for instance N-3: ", probs_3[1])
print("\nPredicted class probabilities for instance N-2: ", class_prob_dict_2)
print("Predicted class for instance N-2: ", probs_2[1])
print("\nPredicted class probabilities for instance N-1: ", class_prob_dict_1)
print("Predicted class for instance N-1: ", probs_1[1])



## Obesity

In [ ]:

# This cell should act as your "main" function where you call the above functions 
# on the full Obesity data set, and print the evaluation score. [0.33 marks]

import numpy as np

# First, read in the data and apply your NB model to the Obesity data
output = preprocess('obesity.csv')
X = output[0]
# After training the dataset, the label is always added to X, so create a copy using np
label = output[1]
X_and_label = output[3]
train_result = train(X,label,X_and_label)
label_prob_dict = train_result[0]
feature_probs = train_result[1]
score = evaluate(X,label,label_prob_dict,feature_probs)

probs_3 = predict(X[-3],label_prob_dict,feature_probs)
# round the number of decimals in the "class probabilities" dictionary
class_prob_dict_3 = probs_3[0]
for key_3 in class_prob_dict_3:
    class_prob_dict_3[key_3] = round(class_prob_dict_3[key_3], 4)

probs_2 = predict(X[-2],label_prob_dict,feature_probs)
# round the number of decimals in the "class probabilities" dictionary
class_prob_dict_2 = probs_2[0]
for key_2 in class_prob_dict_2:
    class_prob_dict_2[key_2] = round(class_prob_dict_2[key_2], 4)

probs_1 = predict(X[-1],label_prob_dict,feature_probs)
# round the number of decimals in the "class probabilities" dictionary
class_prob_dict_1 = probs_1[0]
for key_1 in class_prob_dict_1:
    class_prob_dict_1[key_1] = round(class_prob_dict_1[key_1], 4)



print(score)






# Third, print data statistics and model predictions, as instructed below 
# N is the total number of instances, F the total number of features, L the total number of labels
# The "class probabilities" may be unnormalized


print("Feature vectors of instances [0, 1, 2]: ", X[0:3])

print("\nNumber of instances (N): ", len(X))
print("Number of features (F): ", len(X[0]))
print("Number of labels (L): ",len(set(label)) )

print("\n\nPredicted class probabilities for instance N-3: ", class_prob_dict_3)
print("Predicted class for instance N-3: ", probs_3[1])
print("\nPredicted class probabilities for instance N-2: ",class_prob_dict_2 )
print("Predicted class for instance N-2: ", probs_2[1])
print("\nPredicted class probabilities for instance N-1: ", class_prob_dict_1)
print("Predicted class for instance N-1: ", probs_1[1])






#### Part 2: Conceptual questions [13 marks]

## Question 1: One-R Baseline [3 marks]

In [ ]:
# Write additional code here, if necessary (you may insert additional code cells)
# You should implement the One-R classifier from scratch. Do not use existing implementations/learning algorithms.
# Print the feature name and its corresponding error rate that One-R selects, in addition to any evaluation scores.

# implement baseline model on student and one other dataset
# the second input should be the feature vector of all instances
# the third input is the label
from sklearn.metrics import f1_score
def baseline_classifier(column_names,X,label):
    num_features = len(X[0])
    poss_feat_val = list()
    possible_labels = list(set(label))
    # create a copy of each entry in X appended with its corresponding label
    # the denominator to calculate the error rate is the length of the label list
    denominator = len(label)
    search_space = list()
    for m in range(len(X)):
        temp = X[m]
        temp.append(label[m])
        search_space.append(temp)
    search_space = np.array(search_space)
    for i in range(num_features):
        temp = list(set([instance[i] for instance in X]))
        poss_feat_val.append(temp)
    # initialise a list to remember all labels of all feature values labels
    all_feat_lab = list()
    # initialise a list to remember all error rates
    err_rates = list()
    # go through each feature-value set
    for j in range(len(poss_feat_val)):
        # go through each value in the feature
        running_error = 0
        my_temp = list()
        for k in range(len(poss_feat_val[j])):
            val_label_count_dict = dict()
            # go through each available label
            for indi_lab in possible_labels:
               result = search_space[np.where( (search_space[:,j] == poss_feat_val[j][k]) & (search_space[:,-1] == indi_lab) )]
               register_key = poss_feat_val[j][k] + "_" + indi_lab
               val_label_count_dict[register_key] = len(result)
            # find the maximum key in the in the val_label_count_dict
            max_key = max(val_label_count_dict, key = val_label_count_dict.get)
            underscore_index = max_key.index("_") 
            feat_val = max_key[:underscore_index] 
            max_label = max_key[underscore_index + 1: ] 
            # need to save the max label for each feature value
            my_temp.append({feat_val:max_label})
            # remove the max key from the dictionary
            val_label_count_dict.pop(max_key)
            # get all other values from the dictionary
            values = val_label_count_dict.values()
            total = sum(values) 
            running_error += total
        running_error /= denominator
        err_rates.append(running_error)
        all_feat_lab.append(my_temp) 

    # after all the loops are completed, find the minimum error rate
    min_error = min(err_rates)   
    # find the index of the min err_rates
    index_min_err = err_rates.index(min_error)
    # find the corresponding array with the class labels
    one_r_feat = all_feat_lab[index_min_err]
    best_feature = column_names[index_min_err]
    return (best_feature,min_error,one_r_feat, index_min_err)

# write a small evaluation function
def evaluate_2(X,label,one_r_feat,index_min_err):
    best_feat_test = [instance[index_min_err] for instance in X]
    predictions = list()
    one_r_res = dict()
    for entry in one_r_feat:
        one_r_res.update(entry)
    # generate the predicition for each instance
    for i in range(len(best_feat_test)):
        feature_value = best_feat_test[i]
        temp = one_r_res[feature_value]
        predictions.append(temp)
    # count the number of labels
    number_of_labels = len(list(set(label)))
    label_types = list(set(label))
    evaluation_scores = dict()
    if(number_of_labels == 2):
        evaluation_scores['f1_score'] = f1_score(label,predictions,pos_label=label_types[0],average='binary')
        evaluation_scores['precision'] = precision_score(label,predictions,pos_label=label_types[0],average='binary',zero_division=0)
        evaluation_scores['recall'] = recall_score(label,predictions,pos_label=label_types[0],average='binary',zero_division=0)
    elif(number_of_labels > 2):
        evaluation_scores['f1_score'] = f1_score(label,predictions,average='macro')
        evaluation_scores['precision'] = precision_score(label,predictions,average='macro',zero_division=0)
        evaluation_scores['recall'] = recall_score(label,predictions,average='macro',zero_division=0)

    evaluation_scores['accuracy'] = accuracy_score(label,predictions)
    evaluation_scores['error_rate'] = 1 - evaluation_scores['accuracy']
    return evaluation_scores

# before implementing baseline_classifier, need to implement preprocess function first
# implement the baseline model on student dataset
# get the F1 score macro-average
print('Student: ')
output = preprocess('student.csv')
X = output[0]
label = output[1]
column_names = output[2]
result = baseline_classifier(column_names,X,label)
print("The feature name that one-R selects is: ",result[0],". The corresponding error rate: ", round(result[1],4))
score_1 = evaluate_2(X,label,result[2],result[3])
print(score_1)


print('\nObesity')
output_2 = preprocess('obesity.csv')
X_2 = output_2[0]
label_2 = output_2[1]
column_names_2 = output_2[2]
result_2 = baseline_classifier(column_names_2,X_2,label_2)
score_3 = evaluate_2(X_2,label_2,result_2[2],result_2[3])
print("The feature name that one-R model selects is: ", result_2[0],". The corresponding error rate: ",round(result_2[1],4))
print(score_3)

print('\nBank Marketing')
output_2 = preprocess('bank-marketing.csv')
X_2 = output_2[0]
label_2 = output_2[1]
column_names_2 = output_2[2]
result_2 = baseline_classifier(column_names_2,X_2,label_2)
score_3 = evaluate_2(X_2,label_2,result_2[2],result_2[3])
print("The feature name that one-R model selects is: ", result_2[0],". The corresponding error rate: ",round(result_2[1],4))
print(score_3)


Question 1.b </br>

In the Student Dataset, the error rate of the one-r baseline model was much higher as this model had a rule of majority voting of labels for each attribute value (low,none,mid,high) of the feature Fedu. This is justified by the accuracy of the Naive Bayes model which was 0.4848 (higher than 0.3344) which means more total true positives,recall which was 0.4763 (higher than 0.1896) which means less false negative,precision which was 0.4835(higher than 0.1067) which menas less false positive and f1_score which was 0.4788 (higher than 0.1276), error rate of 0.5161 (lower than 0.6656). The same applies for the Obesity dataset, with the accuracy of its Naive Bayes model being 0.7783 (higher than 0.6652), precision of 0.71 (higher than 0.5585), error rate of 0.221(lower than 0.3647) however the recall which was 0.8765 is lower than the baseline model of (0.9917), suggesting there are more false negative results in the Naive Bayes model. Nonetheless, since most performance parameters of the Naive Bayes model in obesity dataset exceeds the one-line model, we can say the Naive Bayes classifier outperforms the baseline model.

This is because the one-r baseline model lacks complexity and have little predictive power. In the Naive Bayes model, all probabilities of attributes values given the labels was calculated and log aggregated to find the best label whereas only one or the best label corresponding to the attribute value in which its attribute has the lowest error rate, is used in the one-r baseline model. 



## Question 2: Evaluation strategy [3 marks] 


In [ ]:
# Write additional code here, if necessary (you may insert additional code cells)
# cross validation
from itertools import islice
import math
import statistics
from sklearn.metrics import f1_score, accuracy_score

def division_sets(X,label):
    no_blocks = 10
    length_to_split = list()
    block_size = math.floor(len(X) / 10)
    residue_size = len(X) % block_size
    if residue_size == 0:
        length_to_split = [block_size] * 10
    else:
        length_to_split = [block_size] * 9
        length_to_split.append(block_size+residue_size)
    X_copy = iter(X)
    label_copy = iter(label)
    sliced_instances = [list(islice(X_copy,elem)) for elem in length_to_split]
    sliced_label = [list(islice(label_copy,elem)) for elem in length_to_split]
    return (sliced_instances,sliced_label)

def join_train_label(sliced_instances,sliced_label):
    instance_and_label = list()
    for i in range(len(sliced_instances)):
        temp = list(sliced_instances[i])
        temp.append(sliced_label[i])
        instance_and_label.append(temp)
    return instance_and_label
def cross_validation(division_sets,join_train_and_label, X, label):
    sliced_element = division_sets(X,label)
    label_copy = list(label)
    number_of_labels = len(list(set(label)))
    label_types = list(set(label))
    instance = sliced_element[0]
    label = sliced_element[1]
    fones = list()
    precisions = list()
    accuracies = list()
    recalls = list()
    errs = list()
    accuracies = list()
    for i in range(10):
        test_set = instance[i]
        test_label = label[i]
        train_set = list()
        train_label = list()
        if i > 0:
            for m in range(0,i):
                train_set += instance[m]
                train_label += label[m]
                
        for j in range(i+1,10):
            train_set += instance[j]
            train_label += label[j]
        train_set_and_label = join_train_and_label(train_set,train_label)
        param = train(train_set,train_label,train_set_and_label)
        predictions = list()
        for k in range(len(test_set)):
            current_pred = predict(test_set[k],param[0],param[1])
            predictions.append(current_pred[1])
        # calculate the number of label types
       
        if (number_of_labels == 2):
            # remember to append as well
            f1 = f1_score(test_label,predictions,pos_label=label_types[0],average='binary')
            precision = precision_score(test_label,predictions,pos_label=label_types[0],average='binary',zero_division=0)
            recall = recall_score(test_label,predictions,pos_label=label_types[0],average='binary',zero_division=0)
            fones.append(f1)
            precisions.append(precision)
            recalls.append(recall)
        elif(number_of_labels > 2):
            f1 = f1_score(test_label,predictions,average='macro')
            precision = precision_score(test_label,predictions,average='macro',zero_division=0)
            recall = recall_score(test_label,predictions,average='macro',zero_division=0)
            fones.append(f1)
            precisions.append(precision)
            recalls.append(recall)
        accuracy = accuracy_score(test_label,predictions)
        accuracies.append(accuracy)
        err = 1 - accuracy_score(test_label,predictions)
        errs.append(err)
    evaluation_scores = dict()
    evaluation_scores['precision'] = statistics.mean(precisions)
    evaluation_scores['recall'] = statistics.mean(recalls)
    evaluation_scores['f1_score'] = statistics.mean(fones)
    evaluation_scores['accuracy_score'] = statistics.mean(accuracies)
    evaluation_scores['error_rate'] = statistics.mean(errs)
    return evaluation_scores
output_1 = preprocess('bank-marketing.csv')
X_1 = output_1[0]
label_1 = output_1[1]
result = cross_validation(division_sets,join_train_label,X_1,label_1)

print('Bank marketing')
print(result)



# To be used for comparison in Question 4
output_3 = preprocess('student.csv')
X_3 = output_3[0]
label_3 = output_3[1]
result_3 = cross_validation(division_sets,join_train_label,X_3,label_3)
print('\nStudent')
print(result_3)



Question 2 </br>
In the bank marketing dataset, the training error (performing test on training data) (0.1126) is slightly lower than the averaged cross-validation error (0.1137). The precision of cross-validation 0.5166 is much lower than 0.9077 when performing test on training data, suggesting there was a much higher proportion of false positive results in averaged cross-validation. The recall of value of 0.2397 is lower than 0.9715, meaning that the proportion of false negative results is much higher in the averaged cross-validation. There is only a slight difference of accuracy score (0.887 in test on training data and 0.886 in in averaged cross-validation). This is because the bank marketing dataset is imbalanced and most of the instances had the label 'no'. The same justification is applied to the slight difference in error rates as error rate is complementary to accuracy. 

On the other hand, the training error on the Student dataset (0.5161) is much lower than the averaged cross-validation error of 0.6406. The recall of cross validation is 0.2864, which is lower than 0.4763 when testing on training data. The precision  in cross-validation is 0.29 which is lower than 0.4835. 

In conclusion, both the bank marketing and student dataset exhibits lower performance on  cross-validation compared to testing on training data. This is because testing on training data provides a gross overestimation of the Naive Bayes classifier performance. This is because we are effectively telling the classifier what the answers are, while aking the classifier to provide the correct answer. Although all instances acted as both training and testing dataset in cross-validation as well, but the training and testing instances did not overlap in each iteration (fold) of calculating averaged cross-validation.



## Question 3: Feature Selection and Naive Bayes Assumptions [3 marks]

In [ ]:
# Write additional code here, if necessary (you may insert additional code cells)
import pandas as pd
from sklearn import preprocessing
from sklearn.feature_selection import mutual_info_classif
import matplotlib.pyplot as plt
# Mutual Information Obesity and Student Dataset
df = pd.read_csv('obesity.csv')
X = df.iloc[:,1:-1]
x1 = X.apply(preprocessing.LabelEncoder().fit_transform)
y = df.iloc[:, -1]
scores = mutual_info_classif(x1,y,discrete_features=True)
features = df.columns[1:-1]

plt.barh(features,scores)
for index, value in enumerate(scores):
    plt.text(value,index,str(round(value,4)))
plt.ylabel("Obesity features")
plt.xlabel("MI scores")
plt.title("Obesity features with scores")
plt.show()

student_data = pd.read_csv('student.csv')
student_X = student_data.iloc[:,1:-1]
student_x1 = student_X.apply(preprocessing.LabelEncoder().fit_transform)
student_y = student_data.iloc[:,-1]
student_scores = mutual_info_classif(student_x1,student_y,discrete_features=True)

student_features = student_data.columns[1:-1]
plt.barh(student_features,student_scores)
for index,value in enumerate(student_scores):
    plt.text(value,index,str(round(value,4)))
plt.ylabel('Student features')
plt.xlabel('MI scores')
plt.title('Student features with scores')
plt.show()



In [ ]:
from scipy.stats import pearsonr
obesity_X_list = X.values.tolist()
obesity_y_list = y.values.tolist()
obesity_precisions = list()
obesity_recalls = list()
obesity_fones = list()
obesity_accuracies = list()

for m in range(len(obesity_X_list[0])):
    feature = [[instance [m]] for instance in obesity_X_list]
    evaluation_scores = cross_validation(division_sets,join_train_label,feature,obesity_y_list)
    obesity_precisions.append(evaluation_scores['precision'])
    obesity_recalls.append(evaluation_scores['recall'])
    obesity_fones.append(evaluation_scores['f1_score'])
    obesity_accuracies.append(evaluation_scores['accuracy_score'])

student_X_list = student_X.values.tolist()
student_y_list = student_y.values.tolist()

student_precisions = list()
student_recalls = list()
student_fones = list()
student_accuracies = list()
for n in range(len(student_X_list[0])):
     feature = [[instance [n]] for instance in student_X_list]
     student_evaluation = cross_validation(division_sets,join_train_label,feature,student_y_list)
     student_precisions.append(student_evaluation['precision'])
     student_recalls.append(student_evaluation['recall'])
     student_fones.append(student_evaluation['f1_score'])
     student_accuracies.append(student_evaluation['accuracy_score'])

# see how well MI scores correlate to performance
print('Obesity Dataset: ')
plt.figure(0)
plt.scatter(scores,obesity_accuracies)
plt.xlabel('MI Scores')
plt.ylabel('Accuracy')
plt.title('Accuracy vs MI Scores')
corrMI_acc = pearsonr(scores,obesity_accuracies)
print('The correlation between MI scores and accuracy is: ', round(corrMI_acc[0],4))
corrMI_pre = pearsonr(scores,obesity_precisions)
print('The correlation between MI scores and precision is: ',round(corrMI_pre[0],4))
corrMI_rec = pearsonr(scores,obesity_recalls)
print('The correlation between MI scores and recall is: ', round(corrMI_rec[0],4))
corrMI_f1 = pearsonr(scores,obesity_fones)
print('The correlation between MI scores and f_1 is: ', round(corrMI_f1[0],4))
plt.figure(1)
plt.scatter(scores,obesity_precisions)
plt.xlabel('MI Scores')
plt.ylabel('Precision')
plt.title('Precision vs MI Scores')
plt.figure(2)
plt.scatter(scores,obesity_recalls)
plt.xlabel('MI Scores')
plt.ylabel('Recall')
plt.title('Recall vs MI Scores')
plt.figure(3)
plt.scatter(scores,obesity_fones)
plt.xlabel('MI Scores')
plt.ylabel('F1_score')
plt.title('F1_score vs MI Scores')



In [ ]:
print('Student Dataset: ')
plt.figure(0)
plt.scatter(student_scores,student_accuracies)
plt.xlabel('MI scores')
plt.ylabel('Accuracy')
plt.title('Accuracy vs MI scores')
corrMI_acc = pearsonr(student_scores,student_accuracies)
print('The correlation between MI scores and accuracy is: ', round(corrMI_acc[0],1),'.The p-value is: ', round(corrMI_acc[1], 4))

plt.figure(1)
plt.scatter(student_scores,student_precisions)
plt.xlabel('MI scores')
plt.ylabel('Precision')
plt.title('Precision vs MI scores')
corrMI_prec = pearsonr(student_scores,student_precisions)
print('The correlation between MI scores and precision is: ', round(corrMI_prec[0],4),'.The p-value is: ', round(corrMI_prec[1], 4))


plt.figure(2)
plt.scatter(student_scores,student_recalls)
plt.xlabel('MI scores')
plt.ylabel('Recall')
plt.title('Recall vs MI scores')
corrMI_rec = pearsonr(student_scores,student_recalls)
print('The correlation between MI scores and recall is: ', round(corrMI_rec[0],4),'.The p-value is: ', round(corrMI_rec[1], 4))

plt.figure(3)
plt.scatter(student_scores,student_fones)
plt.xlabel('MI scores')
plt.ylabel('F1_score')
plt.title('F1_score vs MI scores')
corrMI_f1 = pearsonr(student_scores,student_fones)
print('The correlation between MI scores and F1_score is: ', round(corrMI_f1[0],4),'.The p-value is: ', round(corrMI_f1[1],4))

Question 3.a </br>
In the Obesity dataset, family_history_with_overweight has the highest MI score and hence highest utility(MI: 0.1105) for the classification task, followed by consumption of food between meals(CAEC) (MI: 0.0819) and then number of main meals (NCP) (0.0402). Some features in the Obesity dataset provides minimal utility to the classification task such as smoke (MI:0.0001) and gender (MI :0). We can say that the MI scores in the Obesity dataset is positively correlated with the utlity/ performance of the classifier because: MI scores has a strong positively correlated relationship with the accuracy (correlation: 0.8578). The same relationship is the same for other evaluation parameters such as recall (correlation: 0.7773) and f1(0.6782). MI score is only moderately postive correlated with precision (correlationL 0.486). family_history_with_overweight and CAEC results in highest utlity in the classification task because the occurrence of these attribute values is highly correlated with the categorical labels that appear in the training dataset. On the other hand, MTRANS and gender are redundant to the dataset, meaning the occurrence of their feature values is completely random and does not correlate with the categorical labels that appear in the dataset. 

In the Student dataset, failures have the highest MI score (MI: 0.115), followed by higher (0.0705)  and Medu (MI: 0.0619) Some features in the Student dataset has lower MI score, such as Pstatus(MI: 0.001), famsize (MI: 0.0022) and sex (MI: 0.0098). We noticed that there is a weak negative relationship between MI scores and accuracy, however since the p-value for this coefficient is greater than 0.05, we can ignore this performance parameter. On the other hand, MI scores have a moderately strong positive relationship with precision (correlatiom: 0.5691), recall(0.5404) and F1-score (0.5105). Hence, this means failure has the highest utility for the model, followed by higher and Medu, whereas famsize has the lowest utility for the model, followed by sex and Pstatus.



Question 3.b </br>
(1) When there are many attributes in the dataset (29 for Student and 13 for Obesity), it would be computationally expensive to calculate P(x1,x2,...,xn | y). To reduce the computation of evaluating this probability, the conditional independence assumption , which is the naivety of the Naive Bayes model is made. This means that conditioned on the class y, the features are assumed to be independent. Also, instances are independent of each other. Stratification should apply in the naivety assumption, meaning the testing data should have the same distribution as the training data.

(2) In the Obesity dataset, the Naive Bayes assumption tells us that given each class (A+, A, B, C, D, F), we treat all features as independent. However, the frequency of consumption of high caloric food (FAVC) is dependent on family_history_of_overweight. This is because the diet habits of an individual is highly influenced by the diet choices of their family. Those who prefer to walk or bike will likely have higher FAF. In the Student dataset, Naive Bayes assumption may be problematic as failures is dependent on the extra educational support provided by the family and the school(famsup & schoolsup).


## Question 4: Feature Selection and Ethics [4 marks]

In [ ]:
# Write additional code here, if necessary (you may insert additional code cells)
import pandas as pd
student_data = pd.read_csv('student.csv')
student_data.head()
student_copy = student_data.drop(labels=['address','Pstatus','Mjob','Fjob','famrel','freetime','internet','health','romantic'],axis=1)
data_list = student_copy.values.tolist()
X = [instance[1:-1] for instance in data_list]
label = [instance[-1] for instance in data_list]
ethical_result = cross_validation(division_sets,join_train_label,X,label)
print('Student Ethical Result Processing')
print(ethical_result)

Question 4.a </br>
One ethical problem that may arise is social bias. The attributes address (urban or rural), MJob (mother's job) and Fjob (father's job) and internet (internet access at home) is highly correlated with socioeconomic status. The classifier may unintentionally be biased against students from low socioeconomic backgrounds. Also, Pstatus (parents cohabitation status) and famrel is related to marital status. Other than that, students may be biased based on their health conditions. Students will also be judged based on their involvement in a romantic relationship, which are all unfair indicators when predicting college grades. 





Question 4.b</br>
There is an general increase in performance compared to when using the full classifier (3 out of 4 performance parameters increased, error rate not considered since it is complementary with accuracy). Using cross-validation evaluation on Student dataset after removing all senstive attributes, the precision increased from 0.29 to 0.30, the recall increased from 0.2864 to 0.2948, the f1_score increased from 0.2657 to 0.2702, the accuracy score decreased from 0.3594 to 0.3514, the error rate increased from 0.6405 to 0.6486. This performance change occurs because the resulting data set carry less biases of the people labelling the data, meaning that the training data became better, which resulted in a better predicition.



Question 4.c </br>
Removing all features in (b) does not guarantee a fair classifier. This is because other variables in the student dataset such as Medu (mother's education), Fedu (father's education), higher (wants to take higher education) may still be highly correlated with the removed attributes. For example, the students decision of continuing with higher education may be affected by the family's available funds to support the student's college tuition instead of their interest in academic persuit.





<b>Authorship Declaration</b>:

   (1) I certify that the program contained in this submission is completely
   my own individual work, except where explicitly noted by comments that
   provide details otherwise.  I understand that work that has been developed
   by another student, or by me in collaboration with other students,
   or by non-students as a result of request, solicitation, or payment,
   may not be submitted for assessment in this subject.  I understand that
   submitting for assessment work developed by or in collaboration with
   other students or non-students constitutes Academic Misconduct, and
   may be penalized by mark deductions, or by other penalties determined
   via the University of Melbourne Academic Honesty Policy, as described
   at https://academicintegrity.unimelb.edu.au.

   (2) I also certify that I have not provided a copy of this work in either
   softcopy or hardcopy or any other form to any other student, and nor will
   I do so until after the marks are released. I understand that providing
   my work to other students, regardless of my intention or any undertakings
   made to me by that other student, is also Academic Misconduct.

   (3) I further understand that providing a copy of the assignment
   specification to any form of code authoring or assignment tutoring
   service, or drawing the attention of others to such services and code
   that may have been made available via such a service, may be regarded
   as Student General Misconduct (interfering with the teaching activities
   of the University and/or inciting others to commit Academic Misconduct).
   I understand that an allegation of Student General Misconduct may arise
   regardless of whether or not I personally make use of such solutions
   or sought benefit from such actions.

   <b>Signed by</b>: Cheah Jia Huei 1078203
   
   <b>Dated</b>: 29/08/2022